# Usage and Development of Post-SCF

## Develop a post-SCF method

Suppose you are going to implement restricted MP2, then you may first run RDH.

Following code gives reference state of B3LYP.

In [1]:
from pyscf import gto
from pyscf.dh import RDHBase, RMP2ofDH, RDH

In [2]:
mol = gto.Mole(atom="O; H 1 0.94; H 1 0.94 2 104.5", basis="6-31G").build()

In [3]:
mf_dh = RDH(mol, xc="B3LYPg").run()

converged SCF energy = -76.3838343073143
[RESULT] Energy of B3LYPG:     -76.383834307314


And then, you just have written a function that solely evaluates MP2 conventional incore correlation energy `kernel_energy_rmp2_conv_full_incore`. You may want to try whether it is correct:

:::{note}

These functions with `kernel_` prefix is encouraged not to call `RDH` instance, or very complicated flags. Make kernel functions as simple as posssible.

Throw most flag-decisions to driver functions, and leave algorithms to kernel functions if possible.

:::

In [4]:
from pyscf.dh.energy.mp2.rmp2 import kernel_energy_rmp2_conv_full_incore

In [5]:
mo_energy = mf_dh.mo_energy
mo_coeff = mf_dh.mo_coeff
nocc, nvir = mf_dh.nocc, mf_dh.nvir

In [6]:
kernel_energy_rmp2_conv_full_incore(mo_energy, mo_coeff, mol, nocc, nvir)


WARN: Conventional integral of MP2 is not recommended!
Use density fitting approximation is recommended.

[RESULT] Energy corr MP2 of same-spin:      -0.0425894017
[RESULT] Energy corr MP2 of oppo-spin:      -0.1399949235
[RESULT] Energy corr MP2 of total:          -0.1825843252


{'eng_corr_MP2_bi1': -0.13999492353172124,
 'eng_corr_MP2_bi2': -0.097405521851971,
 'eng_corr_MP2_OS': -0.13999492353172124,
 'eng_corr_MP2_SS': -0.04258940167975024,
 'eng_corr_MP2': -0.1825843252114715}

After that, you have finished some complicated works on how to manage flags (such as whether a RI or Conventinoal ERI is applied to MP2 correlation) in function `driver_energy_rmp2`.

:::{note}

Make sure the options (such as `integral_scheme_mp2` in the following code) have been defined in `options.py` in any directory of pyscf.dh with default values. Function `pyscf.dh.util.get_default_options` will read these options during importing of `pyscf.dh`.

If you encountered warnings of

> Option {key} is not in the default option list...

then it is recommended to add these options to `options.py`, and restart your python process. Or you can mute warnings.

:::

In [7]:
from pyscf.dh.energy.mp2.rmp2 import driver_energy_rmp2

In [8]:
with mf_dh.params.temporary_flags({"integral_scheme_mp2": "conv"}):
    print("=== Conventional ===")
    print(driver_energy_rmp2(mf_dh))
with mf_dh.params.temporary_flags({"integral_scheme_mp2": "RI"}):
    print("=== RI-MP2 ===")
    print(driver_energy_rmp2(mf_dh))

=== Conventional ===

WARN: Conventional integral of MP2 is not recommended!
Use density fitting approximation is recommended.

[RESULT] Energy corr MP2 of same-spin:      -0.0425894017
[RESULT] Energy corr MP2 of oppo-spin:      -0.1399949235
[RESULT] Energy corr MP2 of total:          -0.1825843252
{'eng_corr_MP2_bi1': -0.13999492353172124, 'eng_corr_MP2_bi2': -0.097405521851971, 'eng_corr_MP2_OS': -0.13999492353172124, 'eng_corr_MP2_SS': -0.04258940167975024, 'eng_corr_MP2': -0.1825843252114715}
=== RI-MP2 ===
[RESULT] Energy corr MP2 of same-spin:      -0.0427631041
[RESULT] Energy corr MP2 of oppo-spin:      -0.1396993715
[RESULT] Energy corr MP2 of total:          -0.1824624756
{'eng_corr_MP2_bi1': -0.1396993715420517, 'eng_corr_MP2_bi2': -0.09693626747354596, 'eng_corr_MP2_OS': -0.1396993715420517, 'eng_corr_MP2_SS': -0.04276310406850574, 'eng_corr_MP2': -0.18246247561055745}


Finally, you have finished the driver function. After `RMP2ofDH.kernel` function finished (which is a must-implemented method, as a wrapper of your driver function), you may try to call your class instance as the following way:

In [9]:
from pyscf.dh.energy.mp2.rmp2 import RMP2ofDH

In [10]:
mf_mp2_dh = RMP2ofDH.from_rdh(mf_dh)

In [11]:
mf_mp2_dh.kernel(integral_scheme_mp2="RI")

[RESULT] Energy corr MP2 of same-spin:      -0.0427631041
[RESULT] Energy corr MP2 of oppo-spin:      -0.1396993715
[RESULT] Energy corr MP2 of total:          -0.1824624756


{'eng_corr_MP2_bi1': -0.1396993715420517,
 'eng_corr_MP2_bi2': -0.09693626747354596,
 'eng_corr_MP2_OS': -0.1396993715420517,
 'eng_corr_MP2_SS': -0.04276310406850574,
 'eng_corr_MP2': -0.18246247561055745}

## Counting energy components

For doubly hybrids (especially xDH framework), energies are explicitly counted by linear combination of several exchange-correlation components (at the given SCF result, B3LYPg for example of XYGn).

Before evaluating exch-corr energies, we may want to evaluate energies other than exch-corr. You may

- Force-run the energy evaluation if the low-rung part of functional is the same to SCF functional (adding option `force_eng_low_rung_revaluate`).
- Evaluating another exch-corr functional.

Then you will get energy other than exch-corr is `'eng_noxc': -67.00438589707584`.

In [12]:
with mf_dh.params.temporary_flags({"force_eng_low_rung_revaluate": True}):
    mf_dh.make_energy_dh("B3LYPg")

[RESULT] Energy of exchange 0.2*HF:      -1.795226773457
[RESULT] Energy of process_energy_low_rung (non_exx): -7.584221636781345
[RESULT] Energy of process_energy_low_rung (total): -9.37944841023839
[RESULT] Energy of B3LYPG:     -76.383834307314


In [13]:
mf_dh.params.results

{'eng_dh_B3LYPG': -76.38383430731429,
 'eng_corr_MP2_bi1': -0.1396993715420517,
 'eng_corr_MP2_bi2': -0.09693626747354596,
 'eng_corr_MP2_OS': -0.1396993715420517,
 'eng_corr_MP2_SS': -0.04276310406850574,
 'eng_corr_MP2': -0.18246247561055745,
 'eng_exx_HF': -8.976133867285224,
 'eng_purexc_B3LYPG': -7.584221636781345,
 'eng_nuc': 9.363261243324963,
 'eng_hcore': -123.35926285877434,
 'eng_j': 46.991615718373495,
 'eng_noxc': -67.00438589707589}

You may also obtain any energy contribution from pure (DFT) xc if you wish.

:::{note}

For B3LYP, the exact exchange part `0.2*HF` is not included in pure part. To evaluate this part, one may run `mf_dh.make_energy_dh("HF")` and obtaining entry `eng_exx_HF`.

`SSR(GGA_X_B88, 0.7)` is scaled short range-separate part of B88 exchange functional, with $\mu = 0.7$.

:::

In [18]:
mf_dh.make_energy_purexc(["GGA_X_B88", ", LYP", "BLYP", "B3LYP", "SSR(GGA_X_B88, 0.7), "])

{'eng_purexc_GGA_X_B88': -9.018238081127627,
 'eng_purexc_, LYP': -0.34043520227989316,
 'eng_purexc_BLYP': -9.358673283407521,
 'eng_purexc_B3LYP': -7.547068290806522,
 'eng_purexc_SSR(GGA_X_B88, 0.7),': -5.844097712132224}

You may also get energy from another functional (using molecular orbitals from B3LYPg):

In [15]:
mf_dh.make_energy_dh("PBE")

[RESULT] Energy of process_energy_low_rung (non_exx): -9.292350024336056
[RESULT] Energy of process_energy_low_rung (total): -9.292350024336056
[RESULT] Energy of PBE:     -76.296735921412


-76.29673592141195

And you may wish to evaluate very complicated functional!

In [16]:
mf_dh.make_energy_dh("0.75*HF + 0.35*LR_HF(1.5) + 0.15*LR_HF(0.75) + 0.25*PBE, 0.5*PBE + 0.5*MP2_OS + 0.15*IEPA(0.6, 0.8) + 0.75*RS_MP2(1.2, 0.85, 0.35)")


WARN: Low-rung DFT has different values of omega found for RSH functional.
We evaluate EXX and pure DFT in separate.
This may cause problem that if some pure DFT components is omega-dependent and our program currently could not handle it.
Anyway, this is purely experimental and use with caution.

[RESULT] Energy of exchange 0.75*HF:      -6.732100400464
[RESULT] Energy of exchange 0.15*LR_HF(0.75):      -0.494072587620
[RESULT] Energy of exchange 0.35*LR_HF(1.5):      -1.714187227123
[RESULT] Energy of process_energy_low_rung (non_exx): -2.4053078553463934
[RESULT] Energy of process_energy_low_rung (total): -2.4053078553463934
[RESULT] Energy of process_energy_low_rung (handle_multiple_omega): -11.3456680705532
[RESULT] Energy of correlation MP2(0.5, 0):      -0.069849685771
[RESULT] Energy of correlation MP2(0.5, 0):      -0.069849685771
[RESULT] Energy of correlation IEPA(0.09, 0.12):      -0.017635729144
[RESULT] Energy corr MP2 of same-spin:      -0.0188505810
[RESULT] Energy corr

-78.46888712235442

All components can be added by its coefficients.

:::{note}

The way of parameters of an advanced correlation, can be viewed in file `pyscf.dh.util.xccode.correlations.definition_corr.json` and classmethod `pyscf.dh.util.XCType.def_parameters`.

For example `RS_MP2(1.2, 0.85, 0.35)`, we found type of `RS_MP2` is `RSMP2`:

```json
    "RS_MP2": {
        "type": "RSMP2",
        "ref": "10.1002/qua.560560417"
    },
```

and found that for type `RSMP2`, there is one essential parameter (range-separate omega) and two optional addable parameters (oppo-spin and same-spin):

```python
            cls.RSMP2: [
                ["range-separate omega", False],
                ["oppo-spin coefficient", True],
                ["same-spin coefficient", True],
            ],
```

As a result, `RS_MP2(1.2, 0.85, 0.35)` refers to $0.85 E^\mathrm{OS}_\mathrm{MP2} (\mu=1.2) + 0.35 E^\mathrm{SS}_\mathrm{MP2} (\mu=1.2)$.

:::

In [17]:
results = mf_dh.params.results

(
    + results["eng_noxc"]
    + 0.75 * results["eng_exx_HF"]
    + 0.35 * results["eng_exx_HF_omega(1.500000)"]
    + 0.15 * results["eng_exx_HF_omega(0.750000)"]
    + results["eng_purexc_0.25*PBE, 0.5*PBE"]
    + 0.5 * results["eng_corr_MP2_OS"]
    + 0.15 * 0.6 * results["eng_corr_IEPA_OS"]
    + 0.15 * 0.8 * results["eng_corr_IEPA_SS"]
    + 0.75 * 0.85 * results["eng_corr_MP2_OS_omega(1.200000)"]
    + 0.75 * 0.35 * results["eng_corr_MP2_SS_omega(1.200000)"]
)

-78.46888712235442

## Adding New Functional

To add a new functional, one may open or create a json file in `pyscf.dh.util.xccode.functionals`.

The essential field for functional dictionary is

- `code`: Energy evaluation functional
- `code_scf`: Self-consistent functional (low-rung only)

For B2PLYP-like functionals (bDH or gDH), the self-consistent functional is the same to energy evaluation functional, except for the advanced correlation (MP2, RPA, etc). For these functionals, `code_scf` can be left empty.

Currently, some fields are defined, but may experience code refactor. Or these fields are not fully utilized in program.

- `ref`: Reference of functional
- `family`: Family of functional (xDH@B3LYP, DSD, revDSD, etc)
- `todo`: Something not fully resolved (such as functionals involving SCAN)

## Adding Tests

See examples in all test directories (not only in `pyscf.dh.tests`, but also in locations like `pyscf.dh.energy.mp2.tests`).